In [ ]:
class CSP:
    def __init__(self, variable, domains, neighbors):

        self.variable = variable
        self.domains = domains
        self.neighbors = neighbors

        self.nodes_expanded = 0
        self.backtracks = 0

    def backtracking_search(self, use_mrv= False):

        self.nodes_expanded = 0
        self.backtracks = 0

        start_time = time.time()
        solution = self._backtrack({}, use_mrv)
        end_time = time.time()

        return {
            'Solution': solution,
            'Time': end_time - start_time,
            'Nodes_Expanded': self.nodes_expanded,
            'Backtracks': self.backtracks
        }

    def _backtrack(self, assignment, use_mrv):

        if len(assignment) == len(self.variable):
            return assignment

        if use_mrv:
            var = self.mrv(assignment)
        else: 
            var = [v for v in self.variable if v not in assignment][0]

        self.nodes_expanded += 1

        for value in self.domains[var]:
            if self.is_consistent(var, value, assignment):
                assignment[var] = value

                result = self._backtrack(assignment, use_mrv)
                if result:
                    return result

                del assignment[var]
                self.backtracks += 1

        return None
    
    def mrv(self, assignment):

        unassigned = [v for v in self.variable if v not in assignment]

        def count_valid_values(var):
            count = 0
            for val in self.domains[var]:
                if self.is_consistent(var, val, assignment):
                    count += 1
            return count

        return min(unassigned, key=count_valid_values)